<center> <h1>Team 1 - Tuesday Batch</h1> </center>
<center> <h3>Aditi Chawla, Aditya Chauhan, Niti Saluja, Zaid Khan</h3> </center>
<center> <h1>Breaking the Social Media </h1> </center>
<center> <h1>News Popularity Prediction</h1> </center>
<br>
<br>
<br>
<center> <h3>IST 718 – Big Data Analytics</h3> </center>
<center> <h3>Daniel E. Acuna</h3> </center>

# Problem & Objectives

- In today’s world most of the people get the current information of current happenings from online news and articles. The current market of online news is large and is growing faster than expected .This leads to tough competition between online publishers and social media platforms in order to reach the largest possible audience.

- One of the important pre-requisites in this industry is to have a highly efficient aligned online strategy based on the trustworthy predictionsof how popular these contents are on different platforms. Given the analysis of the current data we aim at predicting the future popularity of online news given recently published contents .

# Data set Description

- The dataset was downloaded from UCI Machine Learning repository under the link https://archive.ics.uci.edu/ml/datasets/News+Popularity+in+Multiple+Social+Media+Platforms# 
    - The dataset can be downloaded from the "Data Folder" option or by accessing the following link:
    https://archive.ics.uci.edu/ml/machine-learning-databases/00432/Data/
    - The dataset being used is "News_Final.csv"
<br>
- The dataset consists of 12 major columns and has about 93239 records. 
- It consists of data ranging from November 2015 to July 2016 and across various social media platforms- Google plus , Facebook and LinkedIn .It also contains the data about Sentiment Tittle , Topic it was published on before populating on these social media platforms along with publish date. Other columns are sparsely filled and roam around source it was published and Title of article.

# Data Dictionary

- IDLink (numeric): Unique identifier of news items 
- Title (string): Title of the news item according to the official media sources 
- Headline (string): Headline of the news item according to the official media sources 
- Source (string): Original news outlet that published the news item 
- Topic (string): Query topic used to obtain the items in the official media sources 
- PublishDate (timestamp): Date and time of the news items' publication 
- SentimentTitle (numeric): Sentiment score of the text in the news items' title 
- SentimentHeadline (numeric): Sentiment score of the text in the news items' headline 
- Facebook (numeric): Final value of the news items' popularity according to the social media source Facebook 
- GooglePlus (numeric): Final value of the news items' popularity according to the social media source Google+ 
- LinkedIn (numeric): Final value of the news items' popularity according to the social media source LinkedIn

# Packages Used

- from pyspark.sql import **SparkSession**
- from pyspark.ml import **feature, regression, evaluation, Pipeline**
- from pyspark.sql import **functions** as fn, **Row**
- import **matplotlib.pyplot** as plt
- from pyspark.ml import **feature**
- from pyspark.ml import **regression**
- from pyspark.ml import **classification**
- from pyspark.ml import **Pipeline**
- import **pandas** as pd
- import **numpy as** np
- import **seaborn** as sns
- from pyspark.ml.feature import **MaxAbsScaler**
- from pyspark.ml.feature import **Tokenizer**
- from pyspark.ml.feature import **CountVectorizer**
- from pyspark.ml.feature import **VectorAssembler**
- from pyspark.ml.feature import **IDF**
- from pyspark.ml.classification import **LogisticRegression**
- from pyspark.ml.classification import **RandomForestClassifier**
- from pyspark.ml.feature import **IndexToString, StringIndexer, VectorIndexer**
- from pyspark.ml.regression import **RandomForestRegressor**
- from pyspark.ml.feature import **VectorIndexer**
- from pyspark.ml.evaluation import **RegressionEvaluator**
- from pyspark.ml.regression import **GBTRegressor**
- from pyspark.ml.feature import **NGram**
- from pyspark.sql.functions import **col, size**

### Poster
<br>
<center> <img src="Poster.png" width="100%" align="center"> </center>

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
# Functionality for computing features
from pyspark.ml import feature
# Functionality for regression
from pyspark.ml import regression
# Funcionality for classification
from pyspark.ml import classification
# Object for creating sequences of transformations
from pyspark.ml import Pipeline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col, size

###  Reading the Dataset

In [4]:
import pandas as pd
datasource = pd.read_csv('News_Final.csv')

In [5]:
len(datasource)

93239

### Omitting the invalid values
- As the dataset contains -1 values for the popularity score, we shall be omitting those values

In [6]:
datasource = datasource[datasource['LinkedIn'] != -1]

### Creating a User Defined Generic function to check the null values in each column of a pandas dataframe

In [7]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [8]:
missing_values_table(datasource)

Your selected dataframe has 11 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
Source,223,0.3
Headline,14,0.0


In [9]:
datasource.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
6,80690.0,"Monday, 29 Feb 2016","RAMALLAH, February 25, 2016 (WAFA) - Palestine...",NaN,palestine,2016-02-28 14:03:00,0.000000,-0.005906,0,0,0
56,81052.0,"Monday, 29 Feb 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",NaN,palestine,2016-03-01 09:29:00,0.000000,0.048546,0,0,0
111,80994.0,"Tuesday, 1 Mar 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",NaN,palestine,2016-03-01 00:15:00,-0.243068,0.048546,0,0,0
726,311.0,Microsoft offering £100 discount on original B...,Just ahead of the market rollout of Microsoft'...,International Business Times via Yahoo UK & Ir...,microsoft,2015-11-08 05:25:00,0.000000,-0.132812,0,0,0
727,252.0,Economy to improve though no change in last 6 ...,"&quot;In the coming six months, there seems to...",IANS India Private Limited/Yahoo India News vi...,economy,2015-11-08 05:40:00,-0.082022,0.205537,0,0,0


### Checking the description and distribution of the LinkedIn popularity column

In [10]:
datasource['LinkedIn'].describe()

count    87494.000000
mean        17.700185
std        159.381923
min          0.000000
25%          0.000000
50%          0.000000
75%          4.000000
max      20341.000000
Name: LinkedIn, dtype: float64

### Creating a spark dataframe

In [11]:
datasource['Date_Name'] = pd.to_datetime(datasource["PublishDate"]).dt.strftime("%Y%m%d")
datasource[['Title', 'Headline','Source','Topic','PublishDate','Date_Name']] = datasource[['Title', 'Headline','Source','Topic','PublishDate','Date_Name']].astype(str)
datasource.columns = ['IDLInk', 'Title', 'Headline', 'Source', 'Topic', 'PublishDate', 'SentimentTitle', 'SentimentHeadline', 'Facebook', 'GooglePlus', 'LinkedIn','Date_Name']
#datasource.columns
news_df = spark.createDataFrame(datasource)

In [12]:
news_df.columns

['IDLInk',
 'Title',
 'Headline',
 'Source',
 'Topic',
 'PublishDate',
 'SentimentTitle',
 'SentimentHeadline',
 'Facebook',
 'GooglePlus',
 'LinkedIn',
 'Date_Name']

In [13]:
# Converting to Lowercase

news_lower_df = news_df.select('IDLink',fn.lower(fn.col('Title')).alias('Title'),fn.lower(fn.col('Headline')).alias('Headline'),'Source',
                               'Topic','PublishDate', 'SentimentTitle', 'SentimentHeadline', 'Facebook', 'GooglePlus', 'LinkedIn','Date_Name')
                               
news_lower_df.show(2)

+-------+-------------------+--------------------+------+---------+-------------------+--------------+--------------------+--------+----------+--------+---------+
| IDLink|              Title|            Headline|Source|    Topic|        PublishDate|SentimentTitle|   SentimentHeadline|Facebook|GooglePlus|LinkedIn|Date_Name|
+-------+-------------------+--------------------+------+---------+-------------------+--------------+--------------------+--------+----------+--------+---------+
|80690.0|monday, 29 feb 2016|ramallah, februar...|   nan|palestine|2016-02-28 14:03:00|           0.0|-0.00590569489076...|       0|         0|       0| 20160228|
|81052.0|monday, 29 feb 2016|ramallah, februar...|   nan|palestine|2016-03-01 09:29:00|           0.0|  0.0485459183673469|       0|         0|       0| 20160301|
+-------+-------------------+--------------------+------+---------+-------------------+--------------+--------------------+--------+----------+--------+---------+
only showing top 2 row

# Clean Text String

In [14]:
from pyspark.sql.functions import col, lower, regexp_replace, split

def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "^rt ", "")
  c = regexp_replace(c, "(https?\://)\S+", "")
  c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  c = regexp_replace(c,"[\d]+","")  
  #c = split(c, "\\s+") tokenization...
  return c

clean_news_df = news_lower_df.withColumn('Title_Cleaned',clean_text(col('Title')))

clean_news_df.printSchema()
clean_news_df.show(10)

root
 |-- IDLink: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Headline: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- PublishDate: string (nullable = true)
 |-- SentimentTitle: double (nullable = true)
 |-- SentimentHeadline: double (nullable = true)
 |-- Facebook: long (nullable = true)
 |-- GooglePlus: long (nullable = true)
 |-- LinkedIn: long (nullable = true)
 |-- Date_Name: string (nullable = true)
 |-- Title_Cleaned: string (nullable = true)

+-------+--------------------+--------------------+--------------------+---------+-------------------+--------------------+--------------------+--------+----------+--------+---------+--------------------+
| IDLink|               Title|            Headline|              Source|    Topic|        PublishDate|      SentimentTitle|   SentimentHeadline|Facebook|GooglePlus|LinkedIn|Date_Name|       Title_Cleaned|
+-------+--------------------+--------------------+---

# TOKENIZER

In [15]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="Title_Cleaned", outputCol="vector")
title_vector_df = tokenizer.transform(clean_news_df)

title_vector_df.printSchema()
title_vector_df.show(10)

root
 |-- IDLink: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Headline: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- PublishDate: string (nullable = true)
 |-- SentimentTitle: double (nullable = true)
 |-- SentimentHeadline: double (nullable = true)
 |-- Facebook: long (nullable = true)
 |-- GooglePlus: long (nullable = true)
 |-- LinkedIn: long (nullable = true)
 |-- Date_Name: string (nullable = true)
 |-- Title_Cleaned: string (nullable = true)
 |-- vector: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------+--------------------+--------------------+--------------------+---------+-------------------+--------------------+--------------------+--------+----------+--------+---------+--------------------+--------------------+
| IDLink|               Title|            Headline|              Source|    Topic|        PublishDate|      SentimentTitle|   SentimentHeadline|Facebook|

# STOP WORDS REMOVAL

In [16]:
from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

In [17]:
# Specify input/output columns
remover.setInputCol("vector")
remover.setOutputCol("vector_no_stopw")

# Transform existing dataframe with the StopWordsRemover
Titlevector_no_stopw_df = remover.transform(title_vector_df)

# Display
Titlevector_no_stopw_df.printSchema()
Titlevector_no_stopw_df.show()

root
 |-- IDLink: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Headline: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- PublishDate: string (nullable = true)
 |-- SentimentTitle: double (nullable = true)
 |-- SentimentHeadline: double (nullable = true)
 |-- Facebook: long (nullable = true)
 |-- GooglePlus: long (nullable = true)
 |-- LinkedIn: long (nullable = true)
 |-- Date_Name: string (nullable = true)
 |-- Title_Cleaned: string (nullable = true)
 |-- vector: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vector_no_stopw: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------+--------------------+--------------------+--------------------+---------+-------------------+--------------------+--------------------+--------+----------+--------+---------+--------------------+--------------------+--------------------+
| IDLink|               Title|        

# BIGRAMS

In [18]:
from pyspark.ml.feature import NGram

# Define NGram transformer
ngram = NGram(n=2, inputCol="vector_no_stopw", outputCol="bigrams")

# Create bigram_df as a transform of unigram_df using NGram tranformer
production_df = ngram.transform(Titlevector_no_stopw_df)

# Display
production_df.printSchema()
production_df.show()

root
 |-- IDLink: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Headline: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- PublishDate: string (nullable = true)
 |-- SentimentTitle: double (nullable = true)
 |-- SentimentHeadline: double (nullable = true)
 |-- Facebook: long (nullable = true)
 |-- GooglePlus: long (nullable = true)
 |-- LinkedIn: long (nullable = true)
 |-- Date_Name: string (nullable = true)
 |-- Title_Cleaned: string (nullable = true)
 |-- vector: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vector_no_stopw: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bigrams: array (nullable = true)
 |    |-- element: string (containsNull = false)

+-------+--------------------+--------------------+--------------------+---------+-------------------+--------------------+--------------------+--------+----------+--------+---------+---------------

In [19]:
from pyspark.sql.functions import col, size

production_df = production_df.where(size(col("bigrams")) >= 2)

# Display
production_df.printSchema()
production_df.show()

root
 |-- IDLink: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Headline: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- PublishDate: string (nullable = true)
 |-- SentimentTitle: double (nullable = true)
 |-- SentimentHeadline: double (nullable = true)
 |-- Facebook: long (nullable = true)
 |-- GooglePlus: long (nullable = true)
 |-- LinkedIn: long (nullable = true)
 |-- Date_Name: string (nullable = true)
 |-- Title_Cleaned: string (nullable = true)
 |-- vector: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vector_no_stopw: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bigrams: array (nullable = true)
 |    |-- element: string (containsNull = false)

+-------+--------------------+--------------------+--------------------+---------+-------------------+--------------------+--------------------+--------+----------+--------+---------+---------------

### Data Processing and One Hot Encoding

In [ ]:
#ALTERING THE TIME STAMP

production_df2 = production_df.withColumn('Date_Name',production_df["Date_Name"].cast("float") )


#ONE HOT ENCODING

from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Topic", outputCol="Topic_numeric").fit(production_df2)
indexed_df = indexer.transform(production_df2)

#type(indexed_df["Topic_numeric"])


encoder = OneHotEncoderEstimator(
    inputCols=["Topic_numeric"],  
    outputCols=["Topic_vector"]
)


model = encoder.fit(indexed_df)

production_df2 = model.transform(indexed_df)

### Word 2 Vec of Title Column

In [ ]:
Title_Word2Vec = feature.Word2Vec(vectorSize=100, minCount=10, inputCol="vector_no_stopw", outputCol="Title_Vect")

### Data Flow 

<br>
<center> <img src="Data Flow.PNG" width="60%" align="center"> </center>

# LINEAR REGRESSION ON BIGRAMS CREATED FROM TITLE

In [17]:
training_df, validation_df, testing_df = production_df2.randomSplit([0.6, 0.3, 0.1], seed=0 )

In [17]:
count_vectorizer_estimator = CountVectorizer(minTF=1., minDF=5).setInputCol('bigrams').setOutputCol('tf_Title')
count_vectorizer_idf = IDF().setInputCol("tf_Title").setOutputCol("tfidf_Title")
Vector_Assembler = feature.VectorAssembler(inputCols=['tfidf_Title']
                                           , outputCol='features')
tfidf_stages_pipeline = Pipeline(stages=[count_vectorizer_estimator,count_vectorizer_idf,Vector_Assembler])

pipe2_model = Pipeline(stages=[
  tfidf_stages_pipeline,regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')
]).fit(training_df)

In [18]:
prediction_df = pipe2_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)


+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|-18.21342537053431|
|      10| 9.973375817108572|
|       0| 45.59153250468265|
|       0| 4.895033277743437|
|       5|13.076119445521728|
+--------+------------------+
only showing top 5 rows



In [19]:

rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

rmse_df.show()

+-----------------+
|             rmse|
+-----------------+
|184.7765144947246|
+-----------------+



### Linear Regression
- Following Models were executed using Linear Regression

<br>
<center> <img src="LR.PNG" width="60%" align="center"> </center>

- Model 1-TitleTFIDF + Topic Vector(One hot) 
- Model 2 - TitleTFIDF + Topic Vector(Dummy)
- Model 3- TitleTFIDF + Topic Vector(One Hot) + Publish Date 
- Model 4 –Regularization (TitleTFIDF + Topic Vector(One Hot) + Publish Date)
- Model 5 – Regularization(TitleTFIDF + Topic Vector(One Hot)+ Publish Date + Sentiment Title)

#### Model 1-TitleTFIDF + Topic Vector(One hot)

In [29]:

pipe_model = Pipeline(stages=[
  feature.VectorAssembler(inputCols=['Title_tfidf', 'Topic_vector'], outputCol='features'),
  #feature.StandardScaler(withStd=False, withMean=True, inputCol='features', outputCol='features_scaled'),
  regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')  
]).fit(training_df)

In [30]:
prediction_df = pipe_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|-32.33004175140637|
|      10| 6.732539069713411|
|       0|24.849754068440873|
|       0|17.654295709557385|
|       5|-17.90697049743175|
+--------+------------------+
only showing top 5 rows



In [31]:
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

In [32]:
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|130.86717301955426|
+------------------+



#### Model 2 - TitleTFIDF + Topic Vector(Dummy)

In [33]:
dummy_df = news_Title_Proc.select(fn.when(fn.col('Topic')=='obama',1).otherwise(0).alias('Topic_Obama'),
                            fn.when(fn.col('Topic')=='palestine',1).otherwise(0).alias('Topic_Palestine'),
                            fn.when(fn.col('Topic')=='economy',1).otherwise(0).alias('Topic_Economy'),
                                  'Title_tfidf', 'LinkedIn'

                          )
dummy_df.show(20)

+-----------+---------------+-------------+--------------------+--------+
|Topic_Obama|Topic_Palestine|Topic_Economy|         Title_tfidf|LinkedIn|
+-----------+---------------+-------------+--------------------+--------+
|          0|              1|            0|(28249,[814,1294]...|       0|
|          0|              1|            0|(28249,[814,1294]...|       0|
|          0|              1|            0|(28249,[898,2528]...|       0|
|          0|              0|            0|(28249,[2,20,189,...|       0|
|          0|              0|            1|(28249,[1,116,622...|       0|
|          0|              0|            1|(28249,[1,116,622...|       0|
|          0|              0|            0|(28249,[2,71,98,4...|       0|
|          0|              0|            1|(28249,[1,1181,49...|       0|
|          0|              0|            1|(28249,[1,7,342,1...|       0|
|          0|              0|            0|(28249,[2,28,81,7...|       1|
|          0|              0|         

In [35]:
#Linear Regression with Title tfidf and topic with dummy variables
pipe_model = Pipeline(stages=[
  feature.VectorAssembler(inputCols=['Topic_Obama','Topic_Palestine', 'Topic_Economy', 'Title_tfidf'], outputCol='features'),
  #feature.StandardScaler(withStd=False, withMean=True, inputCol='features', outputCol='features_scaled'),
  regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')  
]).fit(training_df)

In [36]:
prediction_df = pipe_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)

+--------+-------------------+
|LinkedIn|         prediction|
+--------+-------------------+
|       0|  75.09928578162172|
|       0| 119.19894872441887|
|       7| 119.19894872441887|
|       0|-40.689224124744435|
|      39| 31.757177713458137|
+--------+-------------------+
only showing top 5 rows



In [37]:
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

In [38]:
rmse_df.show()

+-----------------+
|             rmse|
+-----------------+
|142.7637066211151|
+-----------------+



#### Model 3- TitleTFIDF + Topic Vector(One Hot) + Publish Date

In [41]:
#Linear Regression with Title tfidf, Topic_vector and Publishdate 
#Linear Regression
pipe_model = Pipeline(stages=[
  feature.VectorAssembler(inputCols=['Title_tfidf', 'Topic_vector', 'Date_Name'], outputCol='features'),
  regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')  
]).fit(training_df)

In [42]:
prediction_df = pipe_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)

+--------+-------------------+
|LinkedIn|         prediction|
+--------+-------------------+
|       2| -36.47149463005371|
|      10| 0.8176401569544396|
|       0|  20.37265079719691|
|       0| 13.749502936228964|
|       5|-23.285376369121877|
+--------+-------------------+
only showing top 5 rows



In [43]:
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

In [44]:
rmse_df.show()

+-----------------+
|             rmse|
+-----------------+
|130.8979205266552|
+-----------------+



#### Model 4 –Regularization (TitleTFIDF + Topic Vector(One Hot) + Publish Date)

In [45]:
#Linear Regression with Title tfidf, Topic_vector and Publishdate with regularization 1
#Linear Regression
pipe_model = Pipeline(stages=[
  feature.VectorAssembler(inputCols=['Title_tfidf','Topic_vector' ,'Date_Name'], outputCol='features'),
  #feature.StandardScaler(withStd=False, withMean=True, inputCol='features', outputCol='features_scaled'),
  regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')  
    .setMaxIter(10)
  .setRegParam(0.3)
  .setElasticNetParam(0.8)
]).fit(training_df)

In [46]:
prediction_df = pipe_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)

+--------+-------------------+
|LinkedIn|         prediction|
+--------+-------------------+
|       2| -7.294303640389444|
|      10|-2.4146784262975416|
|       0|   4.09783572267861|
|       0|  11.90262137483478|
|       5| 1.0415837966957042|
+--------+-------------------+
only showing top 5 rows



In [47]:
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

In [48]:
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|110.25568731465081|
+------------------+



#### Model 5 – Regularization(TitleTFIDF + Topic Vector(One Hot)+ Publish Date + Sentiment Title)

In [49]:
#Linear Regression with Title tfidf, Topic_vector and Publishdate with regularization 2
#Linear Regression
pipe_model = Pipeline(stages=[
  feature.VectorAssembler(inputCols=['Title_tfidf','Topic_vector' ,'Date_Name', 'SentimentTitle'], outputCol='features'),
  #feature.StandardScaler(withStd=False, withMean=True, inputCol='features', outputCol='features_scaled'),
  regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')  
    .setMaxIter(10)
  .setRegParam(0.3)
  .setElasticNetParam(0.2)
]).fit(training_df)

In [50]:
prediction_df = pipe_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)

+--------+-------------------+
|LinkedIn|         prediction|
+--------+-------------------+
|       2|-24.727705272176536|
|      10|-0.6823508163779479|
|       0|  10.11249792613853|
|       0| 12.537786014585436|
|       5|-17.003457270870967|
+--------+-------------------+
only showing top 5 rows



In [52]:
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

In [53]:
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|119.31515253397016|
+------------------+



In [54]:
#Linear Regression with Title tfidf, Topic_vector, Publishdate and Sentiment Title without regularization
#Linear Regression
pipe_model = Pipeline(stages=[
  feature.VectorAssembler(inputCols=['Title_tfidf','Topic_vector' ,'Date_Name','SentimentTitle'], outputCol='features'),
  regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')  
]).fit(training_df)

In [55]:
prediction_df = pipe_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)

+--------+-------------------+
|LinkedIn|         prediction|
+--------+-------------------+
|       2|-36.755872137748156|
|      10|0.47840043248834263|
|       0| 20.374118666653885|
|       0| 13.418391002156568|
|       5|-23.497855093584803|
+--------+-------------------+
only showing top 5 rows



In [57]:
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

In [58]:
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|130.89102769356703|
+------------------+



In [59]:
#Final Linear Regression with Title tfidf, topic_vector, Publishdate and Sentiment title with regularization 

pipe_model = Pipeline(stages=[
  feature.VectorAssembler(inputCols=['Title_tfidf','Topic_vector' ,'Date_Name', 'SentimentTitle'], outputCol='features'),
  #feature.StandardScaler(withStd=False, withMean=True, inputCol='features', outputCol='features_scaled'),
  regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')  
    .setMaxIter(20)
  .setRegParam(2.0)
  .setElasticNetParam(1.0)
]).fit(training_df)

In [60]:
prediction_df = pipe_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)

+--------+-----------------+
|LinkedIn|       prediction|
+--------+-----------------+
|       2|9.554063721972184|
|      10|10.66254629135987|
|       0|10.66254629135987|
|       0|10.66254629135987|
|       5|10.66254629135987|
+--------+-----------------+
only showing top 5 rows



In [61]:
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

In [62]:
rmse_df.show()

+-----------------+
|             rmse|
+-----------------+
|96.37281336466918|
+-----------------+



#### Linear Regression Coefficients

<br>
<center> <img src="Coefficient.PNG" width="50%" align="center"> </center>

In [63]:
## Coefficients for the model
coeff=pipe_model.stages[1].coefficients.values

In [64]:
coeff2=coeff[:-3]
coeff3=coeff[-3:]
np.array(coeff3)
import numpy as np
mean_coeffs = np.mean(coeff2)
mean_coeffs

13.680860421441198

In [65]:
final_coeff=np.append(mean_coeffs , coeff3)
final_coeff

array([ 13.68086042, -15.24393408,  -0.99969282,   4.47218909])

#### Cross Validation on Linear Regression

In [66]:
import pyspark.ml.tuning as tune
grid = tune.ParamGridBuilder()

In [15]:
reg = regression.LinearRegression(labelCol = 'LinkedIn', featuresCol = 'features', maxIter=5)

In [16]:
grid = grid.addGrid(reg.elasticNetParam, [0.6, 0.8, 1.0])

In [17]:
grid = grid.addGrid(reg.regParam, np.arange(1,3,.5))

In [18]:
np.arange(1,3,.5)

array([1. , 1.5, 2. , 2.5])

In [19]:
grid = grid.build()

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol=reg.getLabelCol(), predictionCol=reg.getPredictionCol())

In [21]:
va= feature.VectorAssembler(inputCols=['Title_tfidf','Topic_vector' ,'Date_Name', 'SentimentTitle'],outputCol='features')

In [22]:
crossPipe = Pipeline(stages=[va,reg])

In [23]:
cv = tune.CrossValidator(estimator = crossPipe, estimatorParamMaps = grid, evaluator= evaluator, numFolds = 3)

In [24]:
list1=list()

In [25]:
class CrossValidatorVerbose(CrossValidator):
      def _fit(self, dataset):
        est = self.getOrDefault(self.estimator)
        epm = self.getOrDefault(self.estimatorParamMaps)
        numModels = len(epm)

        eva = self.getOrDefault(self.evaluator)
        metricName = eva.getMetricName()

        nFolds = self.getOrDefault(self.numFolds)
        seed = self.getOrDefault(self.seed)
        h = 1.0 / nFolds

        randCol = self.uid + "_rand"
        df = dataset.select("*", rand(seed).alias(randCol))
        metrics = [0.0] * numModels

        for i in range(nFolds):
            foldNum = i + 1
            print("Comparing models on fold %d" % foldNum)
            validateLB = i * h
            validateUB = (i + 1) * h
            condition = (df[randCol] >= validateLB) & (df[randCol] < validateUB)
            validation = df.filter(condition)
            train = df.filter(~condition)

            for j in range(numModels):
                paramMap = epm[j]
                model = est.fit(train, paramMap)
                # TODO: duplicate evaluator to take extra params from input
                metric = eva.evaluate(model.transform(validation, paramMap))
                metrics[j] += metric

                avgSoFar = metrics[j] / foldNum
                print("params: %s\t%s: %f\tavg: %f" % (
                    {param.name: val for (param, val) in paramMap.items()},
                    metricName, metric, avgSoFar))
                list1.append([{param.name: val for (param, val) in paramMap.items()},metric,avgSoFar])
              # paramsList.append([[{param.name: val for (param, val) in paramMap.items()},metricName,metric,avgSoFar]])

        if eva.isLargerBetter():
            bestIndex = np.argmax(metrics)
        else:
            bestIndex = np.argmin(metrics)

        bestParams = epm[bestIndex]
        bestModel = est.fit(dataset, bestParams)
        avgMetrics = [m / nFolds for m in metrics]
        bestAvg = avgMetrics[bestIndex]
        print("Best model:\nparams: %s\t%s: %f" % (
            {param.name: val for (param, val) in bestParams.items()},
            metricName, bestAvg))

        return self._copyValues(CrossValidatorModel(bestModel, avgMetrics))


In [26]:
cvVer = CrossValidatorVerbose(estimator = crossPipe, estimatorParamMaps = grid, evaluator= evaluator, numFolds = 3)

In [27]:
training, test = news_Title_Proc.randomSplit([0.7,0.3],0)

In [28]:
from pyspark.sql.functions import rand
cvVer.fit(training).transform(test)


Comparing models on fold 1
params: {'elasticNetParam': 0.6, 'regParam': 1.0}	rmse: 148.102401	avg: 148.102401
params: {'elasticNetParam': 0.6, 'regParam': 1.5}	rmse: 146.493627	avg: 146.493627
params: {'elasticNetParam': 0.6, 'regParam': 2.0}	rmse: 145.602364	avg: 145.602364
params: {'elasticNetParam': 0.6, 'regParam': 2.5}	rmse: 144.895292	avg: 144.895292
params: {'elasticNetParam': 0.8, 'regParam': 1.0}	rmse: 146.929802	avg: 146.929802
params: {'elasticNetParam': 0.8, 'regParam': 1.5}	rmse: 145.640423	avg: 145.640423
params: {'elasticNetParam': 0.8, 'regParam': 2.0}	rmse: 144.700350	avg: 144.700350
params: {'elasticNetParam': 0.8, 'regParam': 2.5}	rmse: 143.918383	avg: 143.918383
params: {'elasticNetParam': 1.0, 'regParam': 1.0}	rmse: 146.242184	avg: 146.242184
params: {'elasticNetParam': 1.0, 'regParam': 1.5}	rmse: 144.958880	avg: 144.958880
params: {'elasticNetParam': 1.0, 'regParam': 2.0}	rmse: 143.933569	avg: 143.933569
params: {'elasticNetParam': 1.0, 'regParam': 2.5}	rmse: 143.

DataFrame[Date_Name: float, IDLink: double, Title: string, Headline: string, Source: string, Topic: string, Topic_vector: vector, SentimentTitle: double, SentimentHeadline: double, Facebook: bigint, GooglePlus: bigint, LinkedIn: bigint, Title_token: array<string>, Title_Stop: array<string>, Title_tf: vector, Title_tfidf: vector, features: vector, prediction: double]

In [30]:
#Fetch Parameters automatically
list1
df_cross_val=pd.DataFrame(list1,columns=['Regularization_Parameters','RMSE','AVG'])
df_cross_val.head()
df_cross_val['Regularization_Parameters']=df_cross_val['Regularization_Parameters'].astype('str')

In [31]:
df_cross_val['Regularization_Parameters'] = df_cross_val['Regularization_Parameters'].replace({'regParam' : 'regP'}, regex=True)
df_cross_val['Regularization_Parameters'] = df_cross_val['Regularization_Parameters'].replace({'elasticNetParam' : 'elNetP'}, regex=True)
df_cross_val['Regularization_Parameters'] = df_cross_val['Regularization_Parameters'].replace({'{' : ''}, regex=True)
df_cross_val['Regularization_Parameters'] = df_cross_val['Regularization_Parameters'].replace({'}' : ''}, regex=True)

In [32]:
df_cross_val.head(10)

,Regularization_Parameters,RMSE,AVG
0,"'elNetP': 0.6, 'regP': 1.0",148.102401,148.102401
1,"'elNetP': 0.6, 'regP': 1.5",146.493627,146.493627
2,"'elNetP': 0.6, 'regP': 2.0",145.602364,145.602364
3,"'elNetP': 0.6, 'regP': 2.5",144.895292,144.895292
4,"'elNetP': 0.8, 'regP': 1.0",146.929802,146.929802
5,"'elNetP': 0.8, 'regP': 1.5",145.640423,145.640423
6,"'elNetP': 0.8, 'regP': 2.0",144.700350,144.700350
7,"'elNetP': 0.8, 'regP': 2.5",143.918383,143.918383
8,"'elNetP': 1.0, 'regP': 1.0",146.242184,146.242184
9,"'elNetP': 1.0, 'regP': 1.5",144.958880,144.958880


In [33]:
df_cross_val_best=df_cross_val.nsmallest(10, 'RMSE')
df_cross_val_best.head()

,Regularization_Parameters,RMSE,AVG
11,"'elNetP': 1.0, 'regP': 2.5",143.331742,143.331742
7,"'elNetP': 0.8, 'regP': 2.5",143.918383,143.918383
10,"'elNetP': 1.0, 'regP': 2.0",143.933569,143.933569
6,"'elNetP': 0.8, 'regP': 2.0",144.700350,144.700350
3,"'elNetP': 0.6, 'regP': 2.5",144.895292,144.895292


#### Weights derived from Linear Regression

<br>
<center> <img src="Weights.PNG" width="50%" align="center"> </center>

In [71]:
#Final Linear Regression with Title tfidf
pipe_model = Pipeline(stages=[
  feature.VectorAssembler(inputCols=['Title_tfidf'], outputCol='features'),
  regression.LinearRegression(featuresCol='features', labelCol='LinkedIn')  
]).fit(training_df)

In [72]:
prediction_df = pipe_model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)

+--------+-------------------+
|LinkedIn|         prediction|
+--------+-------------------+
|       2| -40.56827288461393|
|      10|  8.621986764496882|
|       0|   22.7774664105211|
|       0| 13.915897851914515|
|       5|-18.387598109956713|
+--------+-------------------+
only showing top 5 rows



In [73]:
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

In [74]:
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|131.00310539927204|
+------------------+



In [76]:
vocabulary = tfidf_Title.stages[2].vocabulary
vocabulary
weights = pipe_model.stages[-1].coefficients.toArray()
weights
coeffs_df = pd.DataFrame({'word': vocabulary, 'weight': weights})

negative_coeffs= coeffs_df.sort_values('weight',ascending=True).head(10)
positive_coeffs = coeffs_df.sort_values('weight',ascending=False).head(10)

In [77]:
positive_coeffs

,word,weight
13931,plumbs,377.488826
8440,approaching,217.782003
20167,bests,200.231914
233,linkedin,153.563009
24439,hurtful,152.550771
5619,queries,146.051192
25769,frightful,114.187867
8932,accounting,109.958851
3043,acquire,108.708127
14880,workflows,108.187819


In [78]:
negative_coeffs

,word,weight
6971,findings,-151.376315
25228,dynamicpoint,-115.176959
18834,huffpo,-101.494627
4434,networking,-87.482485
21169,haggle,-83.781416
28192,dataset,-83.513982
11793,rubbish,-82.852576
24326,surround,-82.389163
12823,symantec,-80.962253
24903,spouts,-80.804652


# RANDOM FOREST REGRESSOR

- Following Models were executed using Random Forest
     
<br>
<center> <img src="Random Forest Models.PNG" width="50%" align="center"> </center>

- Model 1 - Sentiment Title 
- Model 2 – Sentiment Title + Sentiment Headline
- Model 3- Title(TFIDF) 
- Model 4 –TFIDF + Sentiment Title 
- Model 5 – Title(TFIDF) + Sentiment Title + Topic + Publish Date
- Model 6 – (Word2Vec) Title + Sentiment Title +Publish Date + Topic
- Model 7 – Cross Validation(depth=4, bins=5) (Word2Vec) Title + Sentiment Title +Publish Date + Topic

####  Model 1 - Random Forest using Sentiment Title

In [24]:

Vector_Assembler = feature.VectorAssembler(inputCols=['SentimentTitle']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)

rf = RandomForestRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[Vector_Assembler,featureIndexer, rf])
model = pipeline.fit(training_df)



In [25]:

predictions = model.transform(validation_df)

In [28]:
predictions.select('LinkedIn','prediction').show(5)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|19.227798887079537|
|      10|28.521020634001047|
|      12|17.464726886436146|
|      55|14.576659042759378|
|       0| 17.50035243634318|
+--------+------------------+
only showing top 5 rows



In [30]:
evaluator = RegressionEvaluator(
    labelCol="LinkedIn", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on validation data = %g" % rmse)


Root Mean Squared Error (RMSE) on validation data = 128.779


####  Model 2 - Random Forest using Sentiment Title and Sentiment Headline

In [31]:
Vector_Assembler = feature.VectorAssembler(inputCols=['SentimentTitle','SentimentHeadline']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)

rf = RandomForestRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[Vector_Assembler,featureIndexer, rf])
model = pipeline.fit(training_df)

In [32]:
predictions = model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)
evaluator = RegressionEvaluator(
    labelCol="LinkedIn", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on validation data = %g" % rmse)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|19.343776963792152|
|      10| 22.78334902674262|
|      12| 18.08605653903623|
|      55|14.388309185027733|
|       0| 16.79581374381314|
+--------+------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on validation data = 128.847


####  Model 3 - Title Column using TFIDF

In [21]:
count_vectorizer_estimator = CountVectorizer(minTF=1., minDF=5).setInputCol('vector_no_stopw').setOutputCol('tf_Title')
count_vectorizer_idf = IDF().setInputCol("tf_Title").setOutputCol("tfidf_Title")
Vector_Assembler = feature.VectorAssembler(inputCols=['tfidf_Title']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

rf = RandomForestRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[count_vectorizer_estimator,count_vectorizer_idf,Vector_Assembler,featureIndexer, rf])
model = pipeline.fit(training_df)

In [22]:
predictions = model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)


+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|15.014408961346433|
|       0|15.014408961346433|
|       0|15.014408961346433|
|       0|15.014408961346433|
|      12|15.014408961346433|
+--------+------------------+
only showing top 5 rows



In [28]:
predictions.groupby('prediction').count().show()

+------------------+-----+
|        prediction|count|
+------------------+-----+
| 16.55651313003063|    1|
|25.806002264223878|   10|
|14.189645019817643|    1|
|173.83716666487186|    2|
| 17.86705025555253|    2|
| 536.2531276648439|    7|
| 24.29103390870653|   14|
| 14.23638982906312|    3|
| 80.67221771743706|    1|
|1034.9001121903593|    3|
|220.87656214981158|    1|
| 29.99996291094066|    4|
|13.883443028059663|   17|
|1025.5178018283486|    1|
|28.040045896590357|    1|
|17.906262762787822|    1|
| 231.8742415975305|    3|
| 252.8142662496235|    1|
|27.189235300747562|   34|
|103.56431693081515|    1|
+------------------+-----+
only showing top 20 rows



In [23]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+---------------+
|           rmse|
+---------------+
|96.199555384443|
+---------------+



####  Model 4 - Random Forest using Sentiment Title and Title ( TF-IDF )

In [21]:
count_vectorizer_estimator = CountVectorizer(minTF=1., minDF=5).setInputCol('vector_no_stopw').setOutputCol('tf_Title')
count_vectorizer_idf = IDF().setInputCol("tf_Title").setOutputCol("tfidf_Title")
Vector_Assembler = feature.VectorAssembler(inputCols=['tfidf_Title','SentimentTitle']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

rf = RandomForestRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[count_vectorizer_estimator,count_vectorizer_idf,Vector_Assembler,featureIndexer, rf])
model = pipeline.fit(training_df)

In [22]:
predictions = model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|15.337750250341092|
|       0|15.337750250341092|
|       0|15.337750250341092|
|      10|15.337750250341092|
|       0|15.337750250341092|
+--------+------------------+
only showing top 5 rows



In [23]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|122.19902085157491|
+------------------+



#### Model 5 – Title(TFIDF) + Sentiment Title + Topic + Publish Date

In [25]:
count_vectorizer_estimator = CountVectorizer(minTF=1., minDF=5).setInputCol('vector_no_stopw').setOutputCol('tf_Title')
count_vectorizer_idf = IDF().setInputCol("tf_Title").setOutputCol("tfidf_Title")
Vector_Assembler = feature.VectorAssembler(inputCols=['tfidf_Title','Topic_vector','Date_Name','SentimentTitle']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

rf = RandomForestRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[count_vectorizer_estimator,count_vectorizer_idf,Vector_Assembler,featureIndexer, rf])
model = pipeline.fit(training_df)

In [26]:
predictions = model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)


+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|      10|12.815479772988045|
|       0|12.815479772988045|
|      12|12.815479772988045|
|      10|12.815479772988045|
|      55|12.815479772988045|
+--------+------------------+
only showing top 5 rows



In [27]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+-----------------+
|             rmse|
+-----------------+
|176.4165554760131|
+-----------------+



#### Model 6 – (Word2Vec) Title + Sentiment Title +Publish Date + Topic

In [12]:
Vector_Assembler = feature.VectorAssembler(inputCols=['Title_Vect']
                                           , outputCol='features')

featureIndexer =\
    feature.VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

rf = regression.RandomForestRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[Vector_Assembler,featureIndexer, rf])
model = pipeline.fit(training_df)

In [13]:
predictions= model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|12.783914780685615|
|      10| 14.62001131473707|
|       0| 28.05221125411119|
|       0| 19.88761938278107|
|       5|13.187948082097467|
+--------+------------------+
only showing top 5 rows



In [14]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+-----------------+
|             rmse|
+-----------------+
|92.87815713084893|
+-----------------+



#### Model 7 – Cross Validation(depth=4, bins=5) (Word2Vec) Title + Sentiment Title +Publish Date + Topic
- This includes Random Forest using **Cross Validation**

In [12]:
features = ['Title_Vect','SentimentTitle','Date_Name','Topic_vector']  
lr_data = news_Title_Proc.select(fn.col("LinkedIn").alias("label"), *features)  
lr_data.printSchema()  

root
 |-- label: long (nullable = true)
 |-- Title_Vect: vector (nullable = true)
 |-- SentimentTitle: double (nullable = true)
 |-- Date_Name: float (nullable = true)
 |-- Topic_vector: vector (nullable = true)



In [21]:
Vector_Assembler = feature.VectorAssembler(inputCols=features
                                           , outputCol='features')

rf = regression.RandomForestRegressor(labelCol='label',featuresCol="features")
pipeline_cv = Pipeline(stages=[Vector_Assembler, rf])

In [37]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder,CrossValidatorModel
from pyspark.ml.evaluation import RegressionEvaluator

estimatorParam = ParamGridBuilder() \
.addGrid(rf.maxDepth, [4, 6, 8]) \
.addGrid(rf.maxBins, [5, 10, 20, 40]) \
.addGrid(rf.impurity, ["variance"]) \
.build()

evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

In [38]:
from pyspark.sql.functions import rand
list1= list()
class CrossValidatorVerbose(CrossValidator):
  def _fit(self, dataset):
        est = self.getOrDefault(self.estimator)
        epm = self.getOrDefault(self.estimatorParamMaps)
        numModels = len(epm)

        eva = self.getOrDefault(self.evaluator)
        metricName = eva.getMetricName()

        nFolds = self.getOrDefault(self.numFolds)
        seed = self.getOrDefault(self.seed)
        h = 1.0 / nFolds

        randCol = self.uid + "_rand"
        df = dataset.select("*", rand(seed).alias(randCol))
        metrics = [0.0] * numModels

        for i in range(nFolds):
            foldNum = i + 1
            print("Comparing models on fold %d" % foldNum)

            validateLB = i * h
            validateUB = (i + 1) * h
            condition = (df[randCol] >= validateLB) & (df[randCol] < validateUB)
            validation = df.filter(condition)
            train = df.filter(~condition)

            for j in range(numModels):
                paramMap = epm[j]
                model = est.fit(train, paramMap)
                # TODO: duplicate evaluator to take extra params from input
                metric = eva.evaluate(model.transform(validation, paramMap))
                metrics[j] += metric

                avgSoFar = metrics[j] / foldNum
                print("params: %s\t%s: %f\tavg: %f" % (
                    {param.name: val for (param, val) in paramMap.items()},
                    metricName, metric, avgSoFar))
                list1.append([{param.name: val for (param, val) in paramMap.items()},metric,avgSoFar])
              # paramsList.append([[{param.name: val for (param, val) in paramMap.items()},metricName,metric,avgSoFar]])

        if eva.isLargerBetter():
            bestIndex = np.argmax(metrics)
        else:
            bestIndex = np.argmin(metrics)

        bestParams = epm[bestIndex]
        bestModel = est.fit(dataset, bestParams)
        avgMetrics = [m / nFolds for m in metrics]
        bestAvg = avgMetrics[bestIndex]
        print("Best model:\nparams: %s\t%s: %f" % (
            {param.name: val for (param, val) in bestParams.items()},
            metricName, bestAvg))

        return self._copyValues(CrossValidatorModel(bestModel, avgMetrics))

In [39]:
cvVer = CrossValidatorVerbose(estimator = pipeline_cv, estimatorParamMaps = estimatorParam, evaluator= evaluator, numFolds = 3)

In [40]:
train_df,test_df = lr_data.randomSplit([0.8, 0.2], seed=0 )

In [41]:
cvVer.fit(train_df).transform(test_df)

Comparing models on fold 1
params: {'maxDepth': 4, 'maxBins': 5, 'impurity': 'variance'}	rmse: 81.723454	avg: 81.723454
params: {'maxDepth': 4, 'maxBins': 10, 'impurity': 'variance'}	rmse: 82.621209	avg: 82.621209
params: {'maxDepth': 4, 'maxBins': 20, 'impurity': 'variance'}	rmse: 82.747279	avg: 82.747279
params: {'maxDepth': 4, 'maxBins': 40, 'impurity': 'variance'}	rmse: 83.215201	avg: 83.215201
params: {'maxDepth': 6, 'maxBins': 5, 'impurity': 'variance'}	rmse: 85.073498	avg: 85.073498
params: {'maxDepth': 6, 'maxBins': 10, 'impurity': 'variance'}	rmse: 85.339829	avg: 85.339829
params: {'maxDepth': 6, 'maxBins': 20, 'impurity': 'variance'}	rmse: 88.806712	avg: 88.806712
params: {'maxDepth': 6, 'maxBins': 40, 'impurity': 'variance'}	rmse: 90.707984	avg: 90.707984
params: {'maxDepth': 8, 'maxBins': 5, 'impurity': 'variance'}	rmse: 91.347478	avg: 91.347478
params: {'maxDepth': 8, 'maxBins': 10, 'impurity': 'variance'}	rmse: 87.239094	avg: 87.239094
params: {'maxDepth': 8, 'maxBins': 2

DataFrame[label: bigint, Title_Vect: vector, SentimentTitle: double, Date_Name: float, Topic_vector: vector, features: vector, prediction: double]

In [48]:
df_cross_val=pd.DataFrame(list1,columns=['RF_Parameters','RMSE','AVG'])
df_cross_val.head()
df_cross_val['RF_Parameters']=df_cross_val['RF_Parameters'].astype('str')

In [49]:
df_cross_val['RF_Parameters'] = df_cross_val['RF_Parameters'].replace({'maxDepth' : 'Depth'}, regex=True)
df_cross_val['RF_Parameters'] = df_cross_val['RF_Parameters'].replace({'maxBins' : 'Bins'}, regex=True)
df_cross_val['RF_Parameters'] = df_cross_val['RF_Parameters'].replace({'{' : ''}, regex=True)
df_cross_val['RF_Parameters'] = df_cross_val['RF_Parameters'].replace({'}' : ''}, regex=True)

In [50]:
df_cross_val.head(10)

,RF_Parameters,RMSE,AVG
0,"'Depth': 4, 'Bins': 5, 'impurity': 'variance'",81.723454,81.723454
1,"'Depth': 4, 'Bins': 10, 'impurity': 'variance'",82.621209,82.621209
2,"'Depth': 4, 'Bins': 20, 'impurity': 'variance'",82.747279,82.747279
3,"'Depth': 4, 'Bins': 40, 'impurity': 'variance'",83.215201,83.215201
4,"'Depth': 6, 'Bins': 5, 'impurity': 'variance'",85.073498,85.073498
5,"'Depth': 6, 'Bins': 10, 'impurity': 'variance'",85.339829,85.339829
6,"'Depth': 6, 'Bins': 20, 'impurity': 'variance'",88.806712,88.806712
7,"'Depth': 6, 'Bins': 40, 'impurity': 'variance'",90.707984,90.707984
8,"'Depth': 8, 'Bins': 5, 'impurity': 'variance'",91.347478,91.347478
9,"'Depth': 8, 'Bins': 10, 'impurity': 'variance'",87.239094,87.239094


In [51]:
df_cross_val_best=df_cross_val.nsmallest(10, 'RMSE')
df_cross_val_best.head()

,RF_Parameters,RMSE,AVG
0,"'Depth': 4, 'Bins': 5, 'impurity': 'variance'",81.723454,81.723454
1,"'Depth': 4, 'Bins': 10, 'impurity': 'variance'",82.621209,82.621209
2,"'Depth': 4, 'Bins': 20, 'impurity': 'variance'",82.747279,82.747279
3,"'Depth': 4, 'Bins': 40, 'impurity': 'variance'",83.215201,83.215201
4,"'Depth': 6, 'Bins': 5, 'impurity': 'variance'",85.073498,85.073498


# GBT Regressor

- Following Models were executed using GBT Regressor
     
<br>
<center> <img src="GBT Regressor.PNG" width="50%" align="center"> </center>

- Model 1-Title(TFIDF) + Sentiment Title
- Model 2 - Title (TFIDF) + Sentiment Title + Topic(One Hot)
- Model 3- Title (TFIDF)
- Model 4 – Title (TFIDF) +Sentiment Title + Topic (One Hot) + Publish date
- Model 5 – Title(Word2Vect) +Sentiment Title + Topic (One Hot) + Publish date 

#### Model 1-Title(TFIDF) + Sentiment Title

In [23]:
count_vectorizer_estimator = CountVectorizer(minTF=1., minDF=5).setInputCol('vector_no_stopw').setOutputCol('tf_Title')
count_vectorizer_idf = IDF().setInputCol("tf_Title").setOutputCol("tfidf_Title")
Vector_Assembler = feature.VectorAssembler(inputCols=['tfidf_Title','SentimentTitle']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

gbt = GBTRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[count_vectorizer_estimator,count_vectorizer_idf,Vector_Assembler,featureIndexer, gbt])
model = pipeline.fit(training_df)


In [24]:
predictions = model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|11.092685355244646|
|       0|12.202847466714859|
|      10|12.202847466714859|
|     129|12.202847466714859|
|       0|12.202847466714859|
+--------+------------------+
only showing top 5 rows



In [25]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|149.81984260843709|
+------------------+



#### Model 2 - Title (TFIDF) + Sentiment Title + Topic(One Hot)

In [ ]:
count_vectorizer_estimator = CountVectorizer(minTF=1., minDF=5).setInputCol('vector_no_stopw').setOutputCol('tf_Title')
count_vectorizer_idf = IDF().setInputCol("tf_Title").setOutputCol("tfidf_Title")
Vector_Assembler = feature.VectorAssembler(inputCols=['tfidf_Title','SentimentTitle','Topic_vector']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

gbt = GBTRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[count_vectorizer_estimator,count_vectorizer_idf,Vector_Assembler,featureIndexer, gbt])
model = pipeline.fit(training_df)

In [29]:
predictions = model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       0|12.035336330541108|
|       0|23.956569204524165|
|      12|12.035336330541108|
|       4|12.035336330541108|
|       0|12.035336330541108|
+--------+------------------+
only showing top 5 rows



In [30]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|129.05745073120403|
+------------------+



#### Model 3- Title (TFIDF)

In [24]:
count_vectorizer_estimator = CountVectorizer(minTF=1., minDF=5).setInputCol('vector_no_stopw').setOutputCol('tf_Title')
count_vectorizer_idf = IDF().setInputCol("tf_Title").setOutputCol("tfidf_Title")
Vector_Assembler = feature.VectorAssembler(inputCols=['tfidf_Title']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

gbt = GBTRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[count_vectorizer_estimator,count_vectorizer_idf,Vector_Assembler,featureIndexer, gbt])
model = pipeline.fit(training_df)

In [25]:
predictions = model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       0|11.857722340539421|
|       0|11.857722340539421|
|     129|11.857722340539421|
|      46|11.857722340539421|
|       0|11.857722340539421|
+--------+------------------+
only showing top 5 rows



In [26]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+-----------------+
|             rmse|
+-----------------+
|122.5777926633404|
+-----------------+



#### Model 4 – Title (TFIDF) +Sentiment Title + Topic (One Hot) + Publish date

In [29]:
count_vectorizer_estimator = CountVectorizer(minTF=1., minDF=5).setInputCol('vector_no_stopw').setOutputCol('tf_Title')
count_vectorizer_idf = IDF().setInputCol("tf_Title").setOutputCol("tfidf_Title")
Vector_Assembler = feature.VectorAssembler(inputCols=['tfidf_Title','Topic_vector','Date_Name','SentimentTitle']
                                           , outputCol='features')

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

gbt = GBTRegressor(labelCol='LinkedIn',featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[count_vectorizer_estimator,count_vectorizer_idf,Vector_Assembler,featureIndexer, gbt])
model = pipeline.fit(training_df)

In [30]:
predictions = model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)


+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       0|13.063043913033296|
|      46|13.063043913033296|
|      39|13.063043913033296|
|      12|13.063043913033296|
|       0|13.063043913033296|
+--------+------------------+
only showing top 5 rows



In [31]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|101.56705564184892|
+------------------+



#### Model 5 – Title(Word2Vect) +Sentiment Title + Topic (One Hot) + Publish date

In [50]:
#Gradient Boosting Regression
Vector_Assembler = feature.VectorAssembler(inputCols=['Title_Vect','SentimentTitle','Date_Name','Topic_vector']
                                           , outputCol='features')
featureIndexer =\
   feature.VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")
GBT_model = Pipeline(stages=[
Vector_Assembler,featureIndexer,
regression.GBTRegressor(featuresCol='indexedFeatures', labelCol='LinkedIn')]).fit(training_df)

In [53]:
predictions= GBT_model.transform(validation_df)
predictions.select('LinkedIn','prediction').show(5)

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2| 29.62197438318668|
|      10|10.359271496097856|
|       0| 21.60001232652279|
|       0| 16.50451981191586|
|       5|13.241034403553702|
+--------+------------------+
only showing top 5 rows



In [54]:
rmse_df = predictions.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))
rmse_df.show()

+------------------+
|              rmse|
+------------------+
|167.59511130442692|
+------------------+



# BEST MODEL

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# Functionality for computing features
from pyspark.ml import feature
# Functionality for regression
from pyspark.ml import regression
# Funcionality for classification
from pyspark.ml import classification
# Object for creating sequences of transformations
from pyspark.ml import Pipeline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import IDF
from pyspark.ml.classification import LogisticRegression

In [2]:
import pandas as pd
datasource = pd.read_csv('News_Final.csv')

datasource = datasource[datasource['LinkedIn'] != -1]

datasource['Date_Name'] = pd.to_datetime(datasource["PublishDate"]).dt.strftime("%Y%m%d")
datasource[['Title', 'Headline','Source','Topic','PublishDate','Date_Name']] = datasource[['Title', 'Headline','Source','Topic','PublishDate','Date_Name']].astype(str)
datasource.columns = ['IDLInk', 'Title', 'Headline', 'Source', 'Topic', 'PublishDate', 'SentimentTitle', 'SentimentHeadline', 'Facebook', 'GooglePlus', 'LinkedIn','Date_Name']
#datasource.columns
news_df = spark.createDataFrame(datasource)

In [3]:
news_lower_df = news_df.select('IDLink',fn.lower(fn.col('Title')).alias('Title'),fn.lower(fn.col('Headline')).alias('Headline'),'Source',
                               'Topic','PublishDate', 'SentimentTitle', 'SentimentHeadline', 'Facebook', 'GooglePlus', 'LinkedIn','Date_Name')
                               

from pyspark.sql.functions import col, lower, regexp_replace, split

# CLEANING TEXT STRING

def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "^rt ", "")
  c = regexp_replace(c, "(https?\://)\S+", "")
  c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  c = regexp_replace(c,"[\d]+","")  
  #c = split(c, "\\s+") tokenization...
  return c

clean_news_df = news_lower_df.withColumn('Title_Cleaned',clean_text(col('Title')))

#TOKENIZER

from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="Title_Cleaned", outputCol="vector")
title_vector_df = tokenizer.transform(clean_news_df)

#STOP WORD REMOVAL

from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

# Specify input/output columns
remover.setInputCol("vector")
remover.setOutputCol("vector_no_stopw")

# Transform existing dataframe with the StopWordsRemover
Titlevector_no_stopw_df = remover.transform(title_vector_df)


#BIGRAMS

from pyspark.ml.feature import NGram

# Define NGram transformer
ngram = NGram(n=2, inputCol="vector_no_stopw", outputCol="bigrams")

# Create bigram_df as a transform of unigram_df using NGram tranformer
production_df = ngram.transform(Titlevector_no_stopw_df)

from pyspark.sql.functions import col, size

production_df = production_df.where(size(col("bigrams")) >= 2)

#ALTERING THE TIME STAMP

production_df2 = production_df.withColumn('Date_Name',production_df["Date_Name"].cast("float") )


#ONE HOT ENCODING

from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Topic", outputCol="Topic_numeric").fit(production_df2)
indexed_df = indexer.transform(production_df2)

#type(indexed_df["Topic_numeric"])


encoder = OneHotEncoderEstimator(
    inputCols=["Topic_numeric"],  
    outputCols=["Topic_vector"]
)


model = encoder.fit(indexed_df)

production_df2 = model.transform(indexed_df)

Title_Word2Vec = feature.Word2Vec(vectorSize=100, minCount=10, inputCol="vector_no_stopw", outputCol="Title_Vect")

In [4]:
training_df, validation_df = production_df2.randomSplit([0.8, 0.2], seed=0)

In [5]:
training_df.show(2)

+------+--------------------+--------------------+-------------+-------+-------------------+-------------------+------------------+--------+----------+--------+-----------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+
|IDLink|               Title|            Headline|       Source|  Topic|        PublishDate|     SentimentTitle| SentimentHeadline|Facebook|GooglePlus|LinkedIn|  Date_Name|       Title_Cleaned|              vector|     vector_no_stopw|             bigrams|Topic_numeric| Topic_vector|
+------+--------------------+--------------------+-------------+-------+-------------------+-------------------+------------------+--------+----------+--------+-----------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+
|   2.0|to one ceo, 'jane...|"""i believe some...|  MarketWatch|economy|2015-11-09 22:40:10| 0.0376889180722205|-0.177667263629675|      42|     

In [6]:
# With all features
Vector_Assembler = feature.VectorAssembler(inputCols=['Title_Vect','SentimentTitle','Date_Name','Topic_vector']
                                           , outputCol='features')
featureIndexer =\
    feature.VectorIndexer(inputCol="features", outputCol="indexedFeatures").setHandleInvalid("skip")

rf = regression.RandomForestRegressor(labelCol='LinkedIn',featuresCol="features", maxDepth=4, maxBins=5, impurity='variance')
pipeline = Pipeline(stages=[Title_Word2Vec,Vector_Assembler,featureIndexer, rf])
model = pipeline.fit(training_df)

### GETTING INPUT 

In [7]:
from pyspark.sql import Row

In [8]:
raw_data = [Row(Title='Donald Trump fuck stormy daniels', SentimentTitle=-0.451, Topic='Obama',PublishDate='20160801'),
            Row(Title='Palestine is a serious issue', SentimentTitle=-0.22, Topic='Palestine',PublishDate='20170101'),
            Row(Title='Economy is booming right now', SentimentTitle=0.65, Topic='Economy',PublishDate='20161111'),
            Row(Title='Microsoft Conquered GitHub', SentimentTitle=0.75, Topic='Microsoft',PublishDate='20180801')
           ]
testing_df = spark.createDataFrame(raw_data)

In [9]:
testing_df.printSchema()

root
 |-- PublishDate: string (nullable = true)
 |-- SentimentTitle: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Topic: string (nullable = true)



In [10]:
testing_df.show()

+-----------+--------------+--------------------+---------+
|PublishDate|SentimentTitle|               Title|    Topic|
+-----------+--------------+--------------------+---------+
|   20160801|        -0.451|Donald Trump fuck...|    Obama|
|   20170101|         -0.22|Palestine is a se...|Palestine|
|   20161111|          0.65|Economy is boomin...|  Economy|
|   20180801|          0.75|Microsoft Conquer...|Microsoft|
+-----------+--------------+--------------------+---------+



In [11]:
testing_df = testing_df.withColumn('Date_Name',testing_df["PublishDate"].cast("float") )

In [12]:
testing_df.columns

['PublishDate', 'SentimentTitle', 'Title', 'Topic', 'Date_Name']

In [13]:
testing_df = testing_df.select(fn.lower(fn.col('PublishDate')).alias('PublishDate'),fn.lower(fn.col('Title')).alias('Title'),
                               fn.lower(fn.col('Topic')).alias('Topic'),'SentimentTitle','Date_Name')
            

In [14]:
testing_df.show()

+-----------+--------------------+---------+--------------+-----------+
|PublishDate|               Title|    Topic|SentimentTitle|  Date_Name|
+-----------+--------------------+---------+--------------+-----------+
|   20160801|donald trump fuck...|    obama|        -0.451|  2.01608E7|
|   20170101|palestine is a se...|palestine|         -0.22|  2.01701E7|
|   20161111|economy is boomin...|  economy|          0.65|2.0161112E7|
|   20180801|microsoft conquer...|microsoft|          0.75|  2.01808E7|
+-----------+--------------------+---------+--------------+-----------+



In [15]:
# CLEANING TEXT STRING

def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "^rt ", "")
  c = regexp_replace(c, "(https?\://)\S+", "")
  c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  c = regexp_replace(c,"[\d]+","")  
  #c = split(c, "\\s+") tokenization...
  return c

testing_df = testing_df.withColumn('Title_Cleaned',clean_text(col('Title')))


#TOKENIZER

tokenizer = Tokenizer(inputCol="Title_Cleaned", outputCol="vector")
testing_df = tokenizer.transform(testing_df)

#STOP WORD REMOVAL
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

# Specify input/output columns
remover.setInputCol("vector")
remover.setOutputCol("vector_no_stopw")

# Transform existing dataframe with the StopWordsRemover
testing_df = remover.transform(testing_df)


#BIGRAMS

from pyspark.ml.feature import NGram

# Define NGram transformer
ngram = NGram(n=2, inputCol="vector_no_stopw", outputCol="bigrams")

# Create bigram_df as a transform of unigram_df using NGram tranformer
testing_df = ngram.transform(testing_df)

from pyspark.sql.functions import col, size

#testing_df = testing_df.where(size(col("bigrams")) >= 2)

indexer = StringIndexer(inputCol="Topic", outputCol="Topic_numeric").fit(testing_df)
indexed_df = indexer.transform(testing_df)

#type(indexed_df["Topic_numeric"])


encoder = OneHotEncoderEstimator(
    inputCols=["Topic_numeric"],  
    outputCols=["Topic_vector"]
)


model_index = encoder.fit(indexed_df)

testing_df = model_index.transform(indexed_df)

Title_Word2Vec = feature.Word2Vec(vectorSize=100, minCount=10, inputCol="vector_no_stopw", outputCol="Title_Vect")


In [16]:
testing_df.show()

+-----------+--------------------+---------+--------------+-----------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+
|PublishDate|               Title|    Topic|SentimentTitle|  Date_Name|       Title_Cleaned|              vector|     vector_no_stopw|             bigrams|Topic_numeric| Topic_vector|
+-----------+--------------------+---------+--------------+-----------+--------------------+--------------------+--------------------+--------------------+-------------+-------------+
|   20160801|donald trump fuck...|    obama|        -0.451|  2.01608E7|donald trump fuck...|[donald, trump, f...|[donald, trump, f...|[donald trump, tr...|          0.0|(3,[0],[1.0])|
|   20170101|palestine is a se...|palestine|         -0.22|  2.01701E7|palestine is a se...|[palestine, is, a...|[palestine, serio...|[palestine seriou...|          1.0|(3,[1],[1.0])|
|   20161111|economy is boomin...|  economy|          0.65|2.0161112E7|economy i

In [17]:
model.transform(testing_df).select('Title','prediction').show()

+--------------------+------------------+
|               Title|        prediction|
+--------------------+------------------+
|donald trump fuck...|14.697249327751198|
|palestine is a se...| 5.390371327529724|
|economy is boomin...|199.30676798164237|
|microsoft conquer...| 78.87656860148579|
+--------------------+------------------+



In [18]:
production_df2.columns

['IDLink',
 'Title',
 'Headline',
 'Source',
 'Topic',
 'PublishDate',
 'SentimentTitle',
 'SentimentHeadline',
 'Facebook',
 'GooglePlus',
 'LinkedIn',
 'Date_Name',
 'Title_Cleaned',
 'vector',
 'vector_no_stopw',
 'bigrams',
 'Topic_numeric',
 'Topic_vector']

In [19]:
production_df2.show(2)

+-------+-------------------+--------------------+------+---------+-------------------+--------------+--------------------+--------+----------+--------+-----------+-------------+---------------+---------------+---------------+-------------+------------+
| IDLink|              Title|            Headline|Source|    Topic|        PublishDate|SentimentTitle|   SentimentHeadline|Facebook|GooglePlus|LinkedIn|  Date_Name|Title_Cleaned|         vector|vector_no_stopw|        bigrams|Topic_numeric|Topic_vector|
+-------+-------------------+--------------------+------+---------+-------------------+--------------+--------------------+--------+----------+--------+-----------+-------------+---------------+---------------+---------------+-------------+------------+
|80690.0|monday, 29 feb 2016|ramallah, februar...|   nan|palestine|2016-02-28 14:03:00|           0.0|-0.00590569489076...|       0|         0|       0|2.0160228E7| monday  feb |[monday, , feb]|[monday, , feb]|[monday ,  feb]|          3.

In [20]:
prediction_df = model.transform(validation_df).select('LinkedIn','prediction')
prediction_df.show(5)
rmse_df = prediction_df.select((fn.col('prediction')- fn.col('LinkedIn')).alias('e')).\
    select(fn.sqrt(fn.avg(fn.col('e')**2)).alias("rmse"))

rmse_df.show()

+--------+------------------+
|LinkedIn|        prediction|
+--------+------------------+
|       2|14.108441937680325|
|      10| 14.04221663826122|
|     129|13.556403099633673|
|      49|25.319727637287674|
|       3|12.265480549261557|
+--------+------------------+
only showing top 5 rows

+-----------------+
|             rmse|
+-----------------+
|92.06034532259707|
+-----------------+



- **Our best model is Random Forest Regression on Sentiment Title, Topic ( One hot encoding ), Title( Word 2 Vec ) and Date**

# Conclusion

A person could use our platform to input a Title and in return it will
analysis and forecast/predict the popularity of that particular Title based
on inputs with an best of RMSE of 81.72. Further the model could be
improvised by providing more data i.e. data collected over longer period of
time and data spread over more topics.

# Reference

- Data Source - UCI machine learning repository
- Paper - Moniz, N., Torgo, L (2018). Multi-Source Social Feedback of Online News Feeds. Retrieved from
https://archive.ics.uci.edu/ml/datasets/News+Popularity+in+Multiple+Social+Media+Platforms